In [1]:
 current_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis"
 target_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis/Politicians"

In [38]:
import os
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import fasttext
import numpy as np
from sklearn.preprocessing import MinMaxScaler
drive.mount('/content/gdrive')
os.chdir(current_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
ls

 cc.en.300.bin      lilienfeld_baselines.ipynb     Meetings/     'Sound Files'/
 cc.en.300.bin.gz   lilienfeld_samples_df.pickle   Politicians/  'Thesis Reports'/


In [4]:
!pip install fasttext
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English#

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=33206561e27cb8b44c389b9140349b7771107757f9e921db2754faa0676bd4dd
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [9]:
ft = fasttext.load_model('cc.en.300.bin.gz')

In [49]:
import pandas as pd
import pickle

# Load the pickled DataFrame
with open('lilienfeld_samples_df.pickle', 'rb') as f:
    loaded_df = pickle.load(f)

# Now "loaded_df" contains the DataFrame that was previously pickled


In [50]:
loaded_df.head()

presidents  scores  \
0            John F. Kennedy   1.408   
1      Franklin D. Roosevelt   1.079   
2              Ronald Reagan   0.912   
3  William Jefferson Clinton   0.569   
4             George W. Bush   0.391   

                                               texts  
0  In the 1st place I think the problems are more...  
1  Hello Hello Greta I did I called with the fact...  
2  Well She core of course was a major part I had...  
3  Most people think you know that uh except for ...  
4  Well thanks Rudy and thanks George Let me make...

In [51]:
loaded_df

presidents  scores  \
0             John F. Kennedy   1.408   
1       Franklin D. Roosevelt   1.079   
2               Ronald Reagan   0.912   
3   William Jefferson Clinton   0.569   
4              George W. Bush   0.391   
5        Dwight D. Eisenhower   0.297   
6           Lyndon B. Johnson   0.173   
7                 Gerald Ford   0.157   
8           James Earl Carter   0.007   
9               Richard Nixon  -0.544   
10             George H. Bush  -0.619   
11            Harry S. Truman  -0.668   
12             Herbert Hoover  -0.866   

                                                texts  
0   In the 1st place I think the problems are more...  
1   Hello Hello Greta I did I called with the fact...  
2   Well She core of course was a major part I had...  
3   Most people think you know that uh except for ...  
4   Well thanks Rudy and thanks George Let me make...  
5   Well I dont know As far as Vice president Nixo...  
6   I want a couple of maybe 3 of the light brown ...  
7   It was August 1 1974 and uh general Hague who ...  
8   Well there are varying degrees of relationship...  
9   Because the United States is concerned by more...  
10  I dont I honestly Larry I dont I had my chance...  
11  Well I I itll sound as if Im Bragging about th...  
12  Republic after the First World War After the F...

In [47]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch.optim as optim

# Load your DataFrame
df = loaded_df

# Function to get FastText embeddings for a text
def get_fasttext_embeddings(text):
    embeddings = ft.get_sentence_vector(text)
    return embeddings

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale the 'scores' column
df['scores'] = scaler.fit_transform(df[['scores']])

df['embeddings'] = df['texts'].apply(get_fasttext_embeddings)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Define custom PyTorch datasets for training and test
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = df[['embeddings']].values
        self.target = df[['scores']].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.data[idx][0])
        y = torch.FloatTensor(self.target[idx])
        return x, y

# Define a regression model with dropout layers
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, dropout_prob):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Model parameters
input_size = 300  # Size of FastText embeddings
hidden_size1 = 128  # Size of the first hidden layer
hidden_size2 = 64   # Size of the second hidden layer
dropout_prob = 0.5  # Dropout probability
batch_size = 1

# Training parameters
num_epochs = 100
early_stopping_patience = 10  # Number of epochs to wait for improvement
lr = 0.001

# Start cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(df)):
    print(f"Starting fold {fold + 1}")

    # Split data into training and test set
    train_df, test_df = df.iloc[train_index], df.iloc[test_index]

    # Create PyTorch datasets
    train_dataset = MyDataset(train_df)
    test_dataset = MyDataset(test_df)

    # DataLoader setup
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model, optimizer, and loss function
    model = RegressionModel(input_size, hidden_size1, hidden_size2, dropout_prob)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Training loop with early stopping
    best_valid_loss = np.inf
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        total_train_loss = 0
        total_valid_loss = 0

        # Training
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # Validation (in this case, using the test set as validation)
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                predicted = model(inputs)
                valid_loss = criterion(predicted, targets)
                total_valid_loss += valid_loss.item()

        avg_valid_loss = total_valid_loss / len(test_loader)

        print(f'Fold {fold + 1}, Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_valid_loss:.4f}')

        # Check for early stopping
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print(f'Early stopping after {epoch + 1} epochs without improvement.')
            break

    # Evaluate the model on the test set after training
    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            predicted = model(inputs)
            test_loss = criterion(predicted, targets)
            total_test_loss += test_loss.item()

    avg_test_loss = total_test_loss / len(test_loader)
    print(f'Fold {fold + 1} Test Loss: {avg_test_loss:.4f}')

    # Optionally, save model, results, etc.
    # ...

# Final evaluation or additional processing
# ...


Starting fold 1
Fold 1, Epoch [1/100], Train Loss: 0.1972, Valid Loss: 0.0654
Fold 1, Epoch [2/100], Train Loss: 0.1704, Valid Loss: 0.0580
Fold 1, Epoch [3/100], Train Loss: 0.1654, Valid Loss: 0.0591
Fold 1, Epoch [4/100], Train Loss: 0.0881, Valid Loss: 0.0667
Fold 1, Epoch [5/100], Train Loss: 0.1053, Valid Loss: 0.0830
Fold 1, Epoch [6/100], Train Loss: 0.0762, Valid Loss: 0.0960
Fold 1, Epoch [7/100], Train Loss: 0.0978, Valid Loss: 0.1002
Fold 1, Epoch [8/100], Train Loss: 0.1170, Valid Loss: 0.0993
Fold 1, Epoch [9/100], Train Loss: 0.1206, Valid Loss: 0.0961
Fold 1, Epoch [10/100], Train Loss: 0.1122, Valid Loss: 0.0854
Fold 1, Epoch [11/100], Train Loss: 0.0851, Valid Loss: 0.0909
Fold 1, Epoch [12/100], Train Loss: 0.1082, Valid Loss: 0.1111
Early stopping after 12 epochs without improvement.
Fold 1 Test Loss: 0.1111
Starting fold 2
Fold 2, Epoch [1/100], Train Loss: 0.2196, Valid Loss: 0.2158
Fold 2, Epoch [2/100], Train Loss: 0.1868, Valid Loss: 0.1848
Fold 2, Epoch [3/100

In [36]:
print(df["scores"])

0     1.408
1     1.079
2     0.912
3     0.569
4     0.391
5     0.297
6     0.173
7     0.157
8     0.007
9    -0.544
10   -0.619
11   -0.668
12   -0.866
Name: scores, dtype: float64
